In [19]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\amite\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [20]:
data=gutenberg.raw('shakespeare-hamlet.txt')
with open('hamlet.txt','w') as file:
    file.write(data)

In [3]:
with open('hamlet.txt','r') as file:
    text=file.read().lower()

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1

In [5]:
input_sequence=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequence.append(n_gram_sequence)

In [6]:
max_seq_len=max(len(x) for x in input_sequence)

In [7]:
from tensorflow.keras.utils import pad_sequences
import numpy as np

input_sequence=pad_sequences(input_sequence,padding='pre',maxlen=max_seq_len)

In [8]:
x,y=input_sequence[:,:-1],input_sequence[:,-1]

In [9]:
import tensorflow as tf
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=7)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GRU,Embedding,Dropout


In [12]:
model=Sequential()
model.add(Embedding(input_dim=total_words,output_dim=100))
model.add(GRU(200,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(150))
model.add(Dense(total_words,activation='softmax'))
model.build(input_shape=(None, max_seq_len - 1))

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 13, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 13, 200)        │       181,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 200)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 150)            │       158,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4818)           │       727,518 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,548,918 (5.91 MB)

 Trainable params: 1,548,918 (5.91 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [15]:
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),verbose=1,epochs=100
)

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - accuracy: 0.0309 - loss: 7.0465 - val_accuracy: 0.0289 - val_loss: 6.8737
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.0449 - loss: 6.4837 - val_accuracy: 0.0453 - val_loss: 6.8993
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - accuracy: 0.0534 - loss: 6.2452 - val_accuracy: 0.0629 - val_loss: 6.8033
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.0665 - loss: 6.0011 - val_accuracy: 0.0595 - val_loss: 6.8830
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.0772 - loss: 5.7911 - val_accuracy: 0.0629 - val_loss: 6.9619
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.0815 - loss: 5.6002 - val_accuracy: 0.0610 - val_loss: 7.0985
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 29s 45ms/step - accuracy: 0.0889 - loss: 5.4090 - val_accuracy: 0.0573 - val_loss: 7.1264
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.0936 - loss: 5

In [16]:
def predicts_next_word(model,tokenizer,text,max_sequence_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list=token_list[-(max_sequence_len-1):]
    token_list= pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted_word_index=np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word

In [18]:
input_text="Is not this something more then"
print(f"Input Text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predicts_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Predicted Next Word: {next_word}")

Input Text:Is not this something more then
Predicted Next Word: fantasie


In [21]:
model.save('predicts_next_word_LSTM.h5')
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)